# Performance Test Tensorflow with Food101 dataset

- Model  : [EfficientNetB0](https://www.tensorflow.org/api_docs/python/tf/keras/applications/efficientnet/EfficientNetB0) (pretrained on ImageNet)  
- Dataset: [Food 101](https://www.tensorflow.org/datasets/catalog/food101)  
- Running this notebook will download ~4.5GB to your local machine
- You can manually clean up the dataset afterwards with `rm -rf ~/tensorflow_datasets/food101/`
- Code adapted from notebook by mrdbourke: https://github.com/mrdbourke/m1-machine-learning-test/blob/main/01_food101_effnet_benchmark.ipynb

In [ ]:
import os
import tensorflow as tf
import tensorflow_datasets as tfds
import pandas as pd
import numpy as np
from tensorflow.keras import layers
from timeit import default_timer as timer 
from library import log
from loguru import logger
log.configure(logfile="./log/notebook.log")

## GPU and version check

In [ ]:
# Force CPU for testing. 
# Ref: https://github.com/tensorflow/tensorflow/issues/31135
# tf.config.set_visible_devices([], 'GPU')

In [ ]:
devices = tf.config.list_physical_devices()
has_gpu_device = any([d.device_type == "GPU" for d in devices])
if not has_gpu_device:
    logger.error("No GPU device found!")

In [ ]:
tf_version = tf.__version__
if tf_version[0] == 1 or int(tf_version[2]) < 7 :
    logger.warning("Not using a recent version of Tensorflow")

## Setup hyperparameters

In [ ]:
BATCH_SIZE = 32
EPOCHS = 5
DATASET_NAME = "food101"

## Download Data

In [ ]:
(train_data, test_data), ds_info = tfds.load(name=DATASET_NAME,
                                             split=["train", "validation"],
                                             shuffle_files=True,
                                             as_supervised=True,
                                             with_info=True)

## Prepare data

In [ ]:
def preprocess_img(image, label, img_shape=224):
    image = tf.image.resize(image, [img_shape, img_shape])
    return tf.cast(image, tf.float32), label

train_data = train_data.map(map_func=preprocess_img, num_parallel_calls=tf.data.AUTOTUNE)
train_data = train_data.shuffle(buffer_size=1000).batch(batch_size=32).prefetch(buffer_size=tf.data.AUTOTUNE)
test_data = test_data.map(preprocess_img, num_parallel_calls=tf.data.AUTOTUNE)
test_data = test_data.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

## Setup model

In [ ]:
tf.random.set_seed(42)

input_shape = (224, 224, 3)
base_model = tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable = False

inputs = layers.Input(shape=input_shape, name="input_layer")
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D(name="pooling_layer")(x)
class_names = ds_info.features["label"].names
outputs = layers.Dense(len(class_names), activation="softmax")(x)
model = tf.keras.Model(inputs, outputs, name="EfficientNetB0_feature_extract")

model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

## Fit the model

In [ ]:
start_time = timer()
history = model.fit(train_data, 
                    epochs=EPOCHS,
                    steps_per_epoch=len(train_data),
                    validation_data=test_data,
                    validation_steps=len(test_data))
end_time = timer()
total_train_time = end_time - start_time
print(f"\nTotal Train time: {total_train_time:.3f} seconds")

## Results

In [ ]:
time_per_epoch = round(total_train_time/EPOCHS, 3)

print(f"Your time_per_epoch:                   {time_per_epoch}")

# Comparable results source: https://github.com/mrdbourke/m1-machine-learning-test/tree/main/results
# Supplimented with results from https://github.com/dylanhogg
print(f"\nComparable time_per_epoch (sec) results:\n")
print(f"TITAN RTX GPU:                            92")
print(f"Apple M1 Max 10 core, 64GB (dylan):      170")
print(f"Apple M1 Max (mrdbourke):                191")
print(f"Apple M1 Pro:                            297")
print(f"Google Colab (K80 GPU):                  424")
print(f"AMD Radeon Pro 5500M 8 GB (Intel Mac):   606")
print(f"Apple M1 (mrdbourke):                    677")
print(f"Apple M1 (dylan):                        868")
print(f"MBP Intel i7 2.6GHz Quad CPU (dylan):  3,287")

print(f"\nYour training details:\n")
print(f"time_per_epoch    : {time_per_epoch}")
print(f"total_train_time  : {total_train_time}")
print(f"epochs            : {EPOCHS}")
print(f"batch_size        : {BATCH_SIZE}")
print(f"num_train_samples : {len(train_data)*BATCH_SIZE}")
print(f"num_test_samples  : {len(test_data)*BATCH_SIZE}")